## Installation des dépendances

In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install langchain-core langchain langchain-google-genai langchain-community chromadb pypdf pillow google-genai streamlit google-generativeai -qqU
%pip install --upgrade langchain-core langchain langchain-google-genai langchain-community chromadb pypdf pillow google-genai streamlit google-generativeai>=1.00 -qqU


!apt-get install -qq -y nodejs npm > /dev/null
!npm install -g localtunnel --silent > /dev/null

## Bibliothèques et imports

In [ ]:
# Imports Python
import os
import pandas as pd
import re
import time
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict
from typing import List, Tuple, Optional
import pickle
from google.colab import drive, userdata
from IPython.display import Markdown

# Imports LangChain
from langchain import PromptTemplate, hub
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.schema import Document
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma

# Import LangChain Community
from langchain_community.document_loaders import PyPDFLoader

# Imports Google & Google Generative AI
from google import genai
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from google.api_core import exceptions
import google.api_core.exceptions as exceptions


from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from tqdm import tqdm

## Configuration workspace

In [ ]:
drive.mount('/content/drive')
project_path = "/content/drive/My Drive/Projet LO17"
os.chdir(project_path)
print("Dossier courant :", os.getcwd())
print("Contenu du dossier :", os.listdir())

pdf_dir = "compiled_pdfs"
pickle_folder = "pickles"
pickle_docs_path = os.path.join(pickle_folder, "docs.pkl")
pickle_docs_par_fichier_path = os.path.join(pickle_folder, "docs_par_fichier.pkl")
docs = []
docs_par_fichier = defaultdict(list)

if os.path.exists(pickle_docs_path) and os.path.exists(pickle_docs_par_fichier_path):
    with open(pickle_docs_path, "rb") as f:
        docs = pickle.load(f)
    with open(pickle_docs_par_fichier_path, "rb") as f:
        docs_par_fichier = pickle.load(f)
    print("Fichiers PDF chargés depuis le cache.")
else:
    os.makedirs(pickle_folder, exist_ok=True)
    pdf_folder = Path(pdf_dir)
    for pdf_path in pdf_folder.glob("*.pdf"):
        loader = PyPDFLoader(str(pdf_path))
        loaded_docs = loader.load()
        for doc in loaded_docs:
            doc.metadata["source"] = pdf_path.name
            docs.append(doc)
            docs_par_fichier[pdf_path.name].append(doc)

    with open(pickle_docs_path, "wb") as f:
        pickle.dump(docs, f)
    with open(pickle_docs_par_fichier_path, "wb") as f:
        pickle.dump(docs_par_fichier, f)
    print("PDF traités et sauvegardés.")

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
client = genai.Client(api_key=GOOGLE_API_KEY)

# Pour faire fonctionner l'API Google, suivre le tuto :
# https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dossier courant : /content/drive/.shortcut-targets-by-id/1TFnnkvrRwXlEoirjZs24KU8KToPxjWd7/Projet LO17
Contenu du dossier : ['compiled_pdfs', 'app.py', 'chroma_db', 'pickles', 'csv', 'clean_up_test', 'question.gsheet', 'CleanPDFs', 'Rapport RAG.gdoc', '=1.00', 'LO17_Projet.ipynb']
Fichiers PDF chargés depuis le cache.


In [ ]:
target_pdf_filename = "Sales et al. - 2016 - Exploiting academic records for predicting student drop out A case study in Brazilian higher educat.pdf"
pdf_dir = "compiled_pdfs"
target_pdf_path = Path(pdf_dir) / target_pdf_filename
csv_folder = "csv"

if target_pdf_path.exists():
    loader = PyPDFLoader(str(target_pdf_path))
    loaded_docs = loader.load()
    data = []
    for doc in loaded_docs:
        data.append({
            "source": target_pdf_filename,
            "page_content": doc.page_content.strip()
        })

    df = pd.DataFrame(data)
    output_csv_path = os.path.join(csv_folder, "extracted_docs.csv")
    df.to_csv(output_csv_path, index=False)

    print(f"{len(df)} pages extraites et sauvegardées dans '{output_csv_path}'.")
else:
    print(f"Erreur : fichier {target_pdf_filename} non trouvé.")

16 pages extraites et sauvegardées dans 'csv/extracted_docs.csv'.


# prompt pour clean up les données:
clean this text for me by deleting text issued from misinterpreted figures or pictures or graphs, however,
i want to keep all text that explains said figures or graphs, and  also, clean up the text  that's issued from tables into something readable by both humans and LLMs: '''

In [ ]:
# Prompt template to query Gemini
llm_prompt_template = """You are a professional document cleaner specialized in preparing text for both human readability and further LLM processing.

I will provide you with a chunk of raw document text. Your task is to clean and extract only the meaningful narrative content by applying the following rules:
Remove:

    Text that appears to be misinterpreted or garbled output from figures, graphs, or images (e.g., axis labels, chart legends, OCR artifacts).

    Mathematical equations, whether inline or block format.

    Page headers and footers (e.g., repeated titles, page numbers, author names).

    Academic references and citations, especially reference lists typically found at the end of academic papers or embedded in text (e.g., "[12]", "(Smith, 2018)").

Keep:

    All narrative text that explains figures, graphs, or tables.

    Section titles and headings, even if they are standalone.

Output format:

    Return only the cleaned text with no extra commentary, metadata, or formatting beyond the cleaned content.

Input Text Chunk:
{context}"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context'] input_types={} partial_variables={} template='You are a professional document cleaner specialized in preparing text for both human readability and further LLM processing.\n\nI will provide you with a chunk of raw document text. Your task is to clean and extract only the meaningful narrative content by applying the following rules:\nRemove:\n\n    Text that appears to be misinterpreted or garbled output from figures, graphs, or images (e.g., axis labels, chart legends, OCR artifacts).\n\n    Mathematical equations, whether inline or block format.\n\n    Page headers and footers (e.g., repeated titles, page numbers, author names).\n\n    Academic references and citations, especially reference lists typically found at the end of academic papers or embedded in text (e.g., "[12]", "(Smith, 2018)").\n\nKeep:\n\n    All narrative text that explains figures, graphs, or tables.\n\n    Section titles and headings, even if they are standalone.\n\nOutput format:\n\n    

In [ ]:
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

#Clean up d'un seul fichier

In [ ]:
import os
import pandas as pd
import time
from pathlib import Path
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import google.api_core.exceptions as exceptions

# Configuration des dossiers
pdf_dir = "compiled_pdfs"
target_pdf_filename = "Jayaprakash - 2014 - Early Alert of Academically At-Risk Students An Open Source Analytics Initiative"
target_pdf_path = Path(pdf_dir) / f"{target_pdf_filename}.pdf"
csv_folder = "./clean_up_test"

# Créer le dossier de sortie s'il n'existe pas
Path(csv_folder).mkdir(exist_ok=True)

# Vérification de la clé API
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY is None:
    raise ValueError("La clé GOOGLE_API_KEY n'est pas définie dans les variables d'environnement.")

# Configuration de l'API
genai.configure(api_key=GOOGLE_API_KEY)

# Template du prompt pour le nettoyage
llm_prompt_template = """You are a professional document cleaner specialized in preparing text for both human readability and further LLM processing.

I will provide you with a chunk of raw document text. Your task is to clean and extract only the meaningful narrative content by applying the following rules:
Remove:

    Text that appears to be misinterpreted or garbled output from figures, graphs, images, or tables (e.g., axis labels, chart legends, OCR artifacts).

    Mathematical equations, whether inline or block format.

    Page headers and footers (e.g., repeated titles, page numbers, author names).

    Academic references and citations, especially reference lists typically found at the end of academic papers or embedded in text (e.g., "[12]", "(Smith, 2018)").

Keep:

    All narrative text that explains figures, graphs, or tables.

    Section titles and headings, even if they are standalone.

Output format:

    Return only the cleaned text with no extra commentary, metadata, or formatting beyond the cleaned content.

Input Text Chunk:
{context}"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

def clean_text_chunk(text_chunk, chunk_idx, total_chunks):
    """
    Nettoie un chunk de texte en utilisant l'API Gemini avec retry logic
    """
    prompt_text = llm_prompt.format(context=text_chunk)
    max_retries = 10
    base_delay = 5
    cleaned_output_for_chunk = text_chunk
    success = False

    for attempt in range(max_retries):
        try:
            print(f"Traitement du chunk {chunk_idx + 1}/{total_chunks} - Tentative {attempt + 1}")
            model = genai.GenerativeModel('gemini-2.0-flash')
            response = model.generate_content(contents=prompt_text)
            try:
                cleaned_output_for_chunk = response.text.strip()
                success = True
                print(f"✓ Chunk {chunk_idx + 1} traité avec succès")
                break
            except AttributeError:
                print(f"⚠ La réponse de l'API n'avait pas d'attribut .text à la tentative {attempt + 1}")

        except exceptions.ServerError as e:
            retry_delay = min(base_delay * (2 ** attempt), 120)  # Max 2 minutes
            print(f"⚠ Erreur serveur (tentative {attempt + 1}/{max_retries}): {e}")

            if attempt < max_retries - 1:
                print(f"⏳ Attente de {retry_delay} secondes avant nouvelle tentative...")
                time.sleep(retry_delay)
            else:
                print(f"❌ Échec après {max_retries} tentatives pour le chunk {chunk_idx + 1} - Erreur Serveur")

        except exceptions.ResourceExhausted as e:
            retry_delay = min(base_delay * (2 ** attempt), 300)  # Max 5 minutes
            print(f"⚠ Quota épuisé (tentative {attempt + 1}/{max_retries}): {e}")

            if attempt < max_retries - 1:
                print(f"⏳ Attente de {retry_delay} secondes (quota épuisé)...")
                time.sleep(retry_delay)
            else:
                print(f"❌ Quota épuisé après {max_retries} tentatives pour le chunk {chunk_idx + 1}")

        except Exception as e:
            retry_delay = min(base_delay * (1.5 ** attempt), 60)
            print(f"⚠ Erreur inattendue (tentative {attempt + 1}/{max_retries}): {e}")

            if attempt < max_retries - 1:
                print(f"⏳ Attente de {retry_delay:.1f} secondes...")
                time.sleep(retry_delay)
            else:
                print(f"❌ Erreur persistante après {max_retries} tentatives pour le chunk {chunk_idx + 1}")

    return cleaned_output_for_chunk

if target_pdf_path.exists():
    from langchain.document_loaders import PyPDFLoader
    loader = PyPDFLoader(str(target_pdf_path))
    loaded_docs = loader.load()

    data = []
    for doc in loaded_docs:
        data.append({
            "source": target_pdf_filename,
            "page_content": doc.page_content.strip()
        })

    df = pd.DataFrame(data)
    input_csv_path = os.path.join(csv_folder, "extracted_docs.csv")
    df.to_csv(input_csv_path, index=False)
    print(f"{len(df)} pages extraites et sauvegardées dans '{input_csv_path}'.")
else:
    raise FileNotFoundError(f"Fichier PDF {target_pdf_filename} non trouvé dans {pdf_dir}")

df = pd.read_csv(input_csv_path)

cleaned_texts = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    text_chunk = row["page_content"]
    prompt_text = llm_prompt.format(context=text_chunk)
    cleaned_chunk = clean_text_chunk(text_chunk, idx, len(df))
    cleaned_texts.append(cleaned_chunk)

df_result = df.copy()
df_result["cleaned_page_content"] = cleaned_texts
output_csv_path = os.path.join(csv_folder, "extracted_docs_cleaned.csv")
df_result.to_csv(output_csv_path, index=False)
print(f"✅ Nettoyage terminé, résultats sauvegardés dans '{output_csv_path}'.")

16 pages extraites et sauvegardées dans 'csv/extracted_docs.csv'.


  0%|          | 0/16 [00:00<?, ?it/s]

Traitement du chunk 1/16 - Tentative 1


  6%|▋         | 1/16 [00:03<00:49,  3.28s/it]

✓ Chunk 1 traité avec succès
Traitement du chunk 2/16 - Tentative 1


...

 94%|█████████▍| 15/16 [00:54<00:02,  2.54s/it]

✓ Chunk 15 traité avec succès
Traitement du chunk 16/16 - Tentative 1
⚠ Erreur inattendue (tentative 1/10): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '43s'}]}}
⏳ Attente de 5.0 secondes...
Traitem

100%|██████████| 16/16 [01:08<00:00,  4.27s/it]

✓ Chunk 16 traité avec succès


✅ Nettoyage terminé, résultats sauvegardés dans 'csv/extracted_docs_cleaned.csv'.


#clean up de plusieurs documents au meme temps

In [ ]:
import os
import pandas as pd
import time
from pathlib import Path
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Configuration des dossiers
pdf_source_dir = "./CleanPDFs/CleanPDF16"
output_dir = "./clean_up_test"

# Créer le dossier de sortie s'il n'existe pas
Path(output_dir).mkdir(exist_ok=True)

# Vérification de la clé API
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY is None:
    raise ValueError("La clé GOOGLE_API_KEY n'est pas définie dans les variables d'environnement.")

# Configuration de l'API
genai.configure(api_key=GOOGLE_API_KEY)

# Template du prompt pour le nettoyage
llm_prompt_template = """You are a professional document cleaner specialized in preparing text for both human readability and further LLM processing.

I will provide you with a chunk of raw document text. Your task is to clean and extract only the meaningful narrative content by applying the following rules:
Remove:

    Text that appears to be misinterpreted or garbled output from figures, graphs, images, or tables (e.g., axis labels, chart legends, OCR artifacts).

    Mathematical equations, whether inline or block format.

    Page headers and footers (e.g., repeated titles, page numbers, author names).

    Academic references and citations, especially reference lists typically found at the end of academic papers or embedded in text (e.g., "[12]", "(Smith, 2018)").

Keep:

    All narrative text that explains figures, graphs, or tables.

    Section titles and headings, even if they are standalone.

Output format:

    Return only the cleaned text with no extra commentary, metadata, or formatting beyond the cleaned content.

Input Text Chunk:
{context}"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

def clean_text_chunk(text_chunk, chunk_idx, total_chunks, pdf_name):
    """
    Nettoie un chunk de texte en utilisant l'API Gemini avec retry logic
    """
    prompt_text = llm_prompt.format(context=text_chunk)
    max_retries = 10
    base_delay = 5
    cleaned_output_for_chunk = text_chunk
    success = False

    for attempt in range(max_retries):
        try:
            print(f"  [{pdf_name}] Traitement du chunk {chunk_idx + 1}/{total_chunks} - Tentative {attempt + 1}")
            model = genai.GenerativeModel('gemini-2.0-flash-exp')
            response = model.generate_content(prompt_text)
            try:
                cleaned_output_for_chunk = response.text.strip()
                success = True
                print(f"  ✓ [{pdf_name}] Chunk {chunk_idx + 1} traité avec succès")
                break
            except AttributeError:
                print(f"  ⚠ [{pdf_name}] La réponse de l'API n'avait pas d'attribut .text à la tentative {attempt + 1}")

        except genai.types.BlockedPromptException as e:
            print(f"  ⚠ [{pdf_name}] Prompt bloqué (tentative {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                retry_delay = min(base_delay * (1.5 ** attempt), 60)
                print(f"  ⏳ [{pdf_name}] Attente de {retry_delay:.1f} secondes...")
                time.sleep(retry_delay)
            else:
                print(f"  ❌ [{pdf_name}] Prompt persistant bloqué après {max_retries} tentatives pour le chunk {chunk_idx + 1}")

        except genai.types.StopCandidateException as e:
            retry_delay = min(base_delay * (2 ** attempt), 120)
            print(f"  ⚠ [{pdf_name}] Réponse stoppée (tentative {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                print(f"  ⏳ [{pdf_name}] Attente de {retry_delay} secondes avant nouvelle tentative...")
                time.sleep(retry_delay)
            else:
                print(f"  ❌ [{pdf_name}] Échec après {max_retries} tentatives pour le chunk {chunk_idx + 1} - Réponse Stoppée")

        except Exception as e:
            # Gestion des erreurs de quota ou serveur
            if "quota" in str(e).lower() or "resource_exhausted" in str(e).lower():
                retry_delay = min(base_delay * (2 ** attempt), 300)
                print(f"  ⚠ [{pdf_name}] Quota épuisé (tentative {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    print(f"  ⏳ [{pdf_name}] Attente de {retry_delay} secondes (quota épuisé)...")
                    time.sleep(retry_delay)
                else:
                    print(f"  ❌ [{pdf_name}] Quota épuisé après {max_retries} tentatives pour le chunk {chunk_idx + 1}")
            else:
                retry_delay = min(base_delay * (1.5 ** attempt), 60)
                print(f"  ⚠ [{pdf_name}] Erreur inattendue (tentative {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    print(f"  ⏳ [{pdf_name}] Attente de {retry_delay:.1f} secondes...")
                    time.sleep(retry_delay)
                else:
                    print(f"  ❌ [{pdf_name}] Erreur persistante après {max_retries} tentatives pour le chunk {chunk_idx + 1}")

    return cleaned_output_for_chunk

def process_single_pdf(pdf_path):
    """
    Traite un seul fichier PDF et retourne le DataFrame nettoyé
    """
    pdf_name = pdf_path.stem
    print(f"\n🔄 Traitement du PDF: {pdf_name}")

    try:
        # Chargement du PDF
        loader = PyPDFLoader(str(pdf_path))
        loaded_docs = loader.load()

        # Extraction des données
        data = []
        for doc in loaded_docs:
            data.append({
                "source": pdf_path.name,
                "page_content": doc.page_content.strip()
            })

        df = pd.DataFrame(data)
        print(f"  📄 {len(df)} pages extraites de {pdf_name}")

        # Nettoyage des textes
        cleaned_texts = []
        print(f"  🧹 Début du nettoyage...")

        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Nettoyage {pdf_name}"):
            text_chunk = row["page_content"]
            cleaned_chunk = clean_text_chunk(text_chunk, idx, len(df), pdf_name)
            cleaned_texts.append(cleaned_chunk)

        # Création du DataFrame final
        df_result = df.copy()
        df_result["cleaned_page_content"] = cleaned_texts

        return df_result, pdf_name

    except Exception as e:
        print(f"  ❌ Erreur lors du traitement de {pdf_name}: {e}")
        return None, pdf_name

def main():
    """
    Fonction principale pour traiter tous les PDFs
    """
    pdf_source_path = Path(pdf_source_dir)

    # Vérifier que le dossier source existe
    if not pdf_source_path.exists():
        raise FileNotFoundError(f"Le dossier source '{pdf_source_dir}' n'existe pas.")

    # Trouver tous les fichiers PDF
    pdf_files = list(pdf_source_path.glob("*.pdf"))

    if not pdf_files:
        print(f"❌ Aucun fichier PDF trouvé dans '{pdf_source_dir}'")
        return

    print(f"📚 {len(pdf_files)} fichiers PDF trouvés à traiter")

    successful_processes = 0
    failed_processes = 0

    # Traitement de chaque PDF
    for pdf_file in pdf_files:
        try:
            df_result, pdf_name = process_single_pdf(pdf_file)

            if df_result is not None:
                # Sauvegarde du CSV
                output_csv_name = f"cleanedup_{pdf_name}.csv"
                output_csv_path = Path(output_dir) / output_csv_name

                df_result.to_csv(output_csv_path, index=False)
                print(f"  ✅ Résultats sauvegardés dans '{output_csv_path}'")
                successful_processes += 1
            else:
                failed_processes += 1

        except Exception as e:
            print(f"  ❌ Erreur critique pour {pdf_file.name}: {e}")
            failed_processes += 1

    # Résumé final
    print(f"\n📊 RÉSUMÉ DU TRAITEMENT:")
    print(f"  ✅ Succès: {successful_processes} fichiers")
    print(f"  ❌ Échecs: {failed_processes} fichiers")
    print(f"  📁 Résultats sauvegardés dans '{output_dir}'")

if __name__ == "__main__":
    main()

FileNotFoundError: Le dossier source './CleanPDFs/CleanPDF16' n'existe pas.

In [ ]:
import os
import pandas as pd
import time
import pickle
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
import google.generativeai as genai

# Configuration des dossiers
pdf_source_dir = "qlq_tests_pdf"
output_dir = "clean_up_test"

# Créer le dossier de sortie s'il n'existe pas
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Vérification de la clé API
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY is None:
    raise ValueError("La clé GOOGLE_API_KEY n'est pas définie dans les variables d'environnement.")

# Configurez l'API key
genai.configure(api_key=GOOGLE_API_KEY)

# Initialisez le modèle
model = genai.GenerativeModel('gemini-2.0-flash')

# Template du prompt pour le nettoyage
llm_prompt_template = """You are a professional document cleaner specialized in preparing text for both human readability and further LLM processing.

I will provide you with a chunk of raw document text. Your task is to clean and extract only the meaningful narrative content by applying the following rules:
Remove:
    Text that appears to be misinterpreted or garbled output from figures, graphs, images, or tables (e.g., axis labels, chart legends, OCR artifacts).
    Mathematical equations, whether inline or block format.
    Page headers and footers (e.g., repeated titles, page numbers, author names).
    Academic references and citations, especially reference lists typically found at the end of academic papers or embedded in text (e.g., "[12]", "(Smith, 2018)").

Keep:
    I want
    All narrative text that explains figures, graphs, or tables.
    Section titles and headings, even if they are standalone.

Output format:
    Return only the cleaned text with no extra commentary, metadata, or formatting beyond the cleaned content.

Input Text Chunk:
{context}
"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

def clean_text_chunk(text_chunk, chunk_idx, total_chunks, pdf_name):
    """
    Nettoie un chunk de texte en utilisant l'API Gemini avec retry logic
    """
    prompt_text = llm_prompt.format(context=text_chunk)
    max_retries = 10
    base_delay = 5
    cleaned_output_for_chunk = text_chunk
    success = False

    for attempt in range(max_retries):
        try:
            print(f"  [{pdf_name}] Traitement du chunk {chunk_idx + 1}/{total_chunks} - Tentative {attempt + 1}")
            response = model.generate_content(contents=prompt_text)
            try:
                cleaned_output_for_chunk = response.text.strip()
                success = True
                print(f"  ✓ [{pdf_name}] Chunk {chunk_idx + 1} traité avec succès")
                break
            except AttributeError:
                print(f"  ⚠ [{pdf_name}] La réponse de l'API n'avait pas d'attribut .text à la tentative {attempt + 1}")

        except Exception as e:
            retry_delay = min(base_delay * (1.5 ** attempt), 60)
            print(f"  ⚠ [{pdf_name}] Erreur (tentative {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                print(f"  ⏳ [{pdf_name}] Attente de {retry_delay:.1f} secondes...")
                time.sleep(retry_delay)
            else:
                print(f"  ❌ [{pdf_name}] Erreur persistante après {max_retries} tentatives pour le chunk {chunk_idx + 1}")

    return cleaned_output_for_chunk

def process_single_pdf(pdf_path):
    """
    Traite un seul fichier PDF et retourne le DataFrame nettoyé
    """
    pdf_name = pdf_path.stem
    print(f"\n🔄 Traitement du PDF: {pdf_name}")

    try:
        # Chargement du PDF
        loader = PyPDFLoader(str(pdf_path))
        loaded_docs = loader.load()

        # Extraction des données
        data = []
        for doc in loaded_docs:
            data.append({
                "source": pdf_path.name,
                "page_content": doc.page_content.strip()
            })

        df = pd.DataFrame(data)
        print(f"  📄 {len(df)} pages extraites de {pdf_name}")

        # Nettoyage des textes
        cleaned_texts = []
        print(f"  🧹 Début du nettoyage...")

        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Nettoyage {pdf_name}"):
            text_chunk = row["page_content"]
            cleaned_chunk = clean_text_chunk(text_chunk, idx, len(df), pdf_name)
            cleaned_texts.append(cleaned_chunk)

        # Création du DataFrame final
        df_result = df.copy()
        df_result["cleaned_page_content"] = cleaned_texts

        return df_result, pdf_name

    except Exception as e:
        print(f"  ❌ Erreur lors du traitement de {pdf_name}: {e}")
        return None, pdf_name

def main():
    """
    Fonction principale pour traiter tous les PDFs
    """
    pdf_source_path = Path(pdf_source_dir)

    # Vérifier que le dossier source existe
    if not pdf_source_path.exists():
        raise FileNotFoundError(f"Le dossier source '{pdf_source_dir}' n'existe pas.")

    # Trouver tous les fichiers PDF
    pdf_files = list(pdf_source_path.glob("*.pdf"))

    if not pdf_files:
        print(f"❌ Aucun fichier PDF trouvé dans '{pdf_source_dir}'")
        return

    print(f"📚 {len(pdf_files)} fichiers PDF trouvés à traiter")

    successful_processes = 0
    failed_processes = 0

    # Traitement de chaque PDF
    for pdf_file in pdf_files:
        try:
            df_result, pdf_name = process_single_pdf(pdf_file)

            if df_result is not None:
                # Sauvegarde du CSV
                output_csv_name = f"cleanedup_{pdf_name}.csv"
                output_csv_path = Path(output_dir) / output_csv_name

                df_result.to_csv(output_csv_path, index=False)
                print(f"  ✅ Résultats sauvegardés dans '{output_csv_path}'")
                successful_processes += 1
            else:
                failed_processes += 1

        except Exception as e:
            print(f"  ❌ Erreur critique pour {pdf_file.name}: {e}")
            failed_processes += 1

    # Résumé final
    print(f"\n📊 RÉSUMÉ DU TRAITEMENT:")
    print(f"  ✅ Succès: {successful_processes} fichiers")
    print(f"  ❌ Échecs: {failed_processes} fichiers")
    print(f"  📁 Résultats sauvegardés dans '{output_dir}'")

if __name__ == "__main__":
    main()


#affichage d'autres trucs

In [ ]:
# Affichage simple des 5 premiers documents dans docs
print("Affichage des 5 premiers documents dans docs :\n")
for i, doc in enumerate(docs[:5]):
    print(f"📄 Document {i + 1} (source: {doc.metadata.get('source', 'inconnu')})")
    print(doc.page_content[:300])
    print("---\n")

In [ ]:
# Affichage du nombre de documents par fichier dans docs_par_fichier
print("\nNombre de documents chargés par fichier (dans docs_par_fichier) :\n")
for filename, doc_list in list(docs_par_fichier.items())[:5]:  # limite à 5 fichiers pour la lisibilité
    print(f"Fichier: {filename} -> {len(doc_list)} documents")
    # Affiche un extrait du premier document pour vérif rapide
    if doc_list:
        print(f"Premier document extrait:\n{doc_list[0].page_content[:200]}")
    print("---\n")

In [ ]:
for i, doc in enumerate(docs[:20]):
    print(f"📄 Document {i + 1}")
    print(doc.page_content[:500])
    print("---\n")

### Ci-dessous un exemple pour tester que tout est bien paramétré :

In [ ]:
response = client.models.generate_content(model=MODEL_ID, contents="Dis qqch, n'importe quoi, je veux apprendre un fait nouveau.")

display(Markdown(response.text))




# Traitement de notre base de documents (à faire)

In [ ]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vectorstore = Chroma.from_documents(
    documents=docs,                 # Les documents chargés
    embedding=gemini_embeddings,    # Modèle d'embedding
    persist_directory="./chroma_db" # Emplacement de la base de données
)

vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",       # Directory of db
                        embedding_function=gemini_embeddings   # Embedding model
                   )

In [ ]:
retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 1})

## aller récupérer les chunks de texte cleaned up et les rajoute à notre base de vecteur

In [ ]:
import os
import pandas as pd
from pathlib import Path
from typing import List
from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from tqdm import tqdm

def load_cleaned_csvs_to_chroma(csv_dir: str = "clean_up_test",
                               chroma_dir: str = "chroma_db",
                               batch_size: int = 50):
    """
    Charge tous les fichiers CSV nettoyés du dossier csv_dir vers ChromaDB

    Args:
        csv_dir: Répertoire contenant les fichiers CSV nettoyés
        chroma_dir: Répertoire de la base de données ChromaDB
        batch_size: Nombre de documents à traiter par batch
    """

    print("🔄 Initialisation du chargement des CSV vers ChromaDB...")

    # Configuration des embeddings
    gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Vérifier si le dossier CSV existe
    csv_path = Path(csv_dir)
    if not csv_path.exists():
        raise FileNotFoundError(f"Le dossier '{csv_dir}' n'existe pas.")

    # Trouver tous les fichiers CSV
    csv_files = list(csv_path.glob("*.csv"))

    if not csv_files:
        print(f"❌ Aucun fichier CSV trouvé dans '{csv_dir}'")
        return None

    print(f"📁 {len(csv_files)} fichiers CSV trouvés")

    # Collecter tous les documents
    all_documents = []
    total_chunks = 0

    for csv_file in csv_files:
        print(f"\n📄 Traitement de {csv_file.name}...")

        try:
            # Charger le CSV
            df = pd.read_csv(csv_file)

            # Vérifier que les colonnes nécessaires existent
            if 'cleaned_page_content' not in df.columns:
                print(f"  ⚠️ Colonne 'cleaned_page_content' manquante dans {csv_file.name}")
                continue

            if 'source' not in df.columns:
                print(f"  ⚠️ Colonne 'source' manquante dans {csv_file.name}")
                continue

            # Créer les documents pour chaque ligne
            file_documents = []
            for idx, row in df.iterrows():
                cleaned_text = row['cleaned_page_content']

                # Ignorer les chunks vides ou très courts
                if pd.isna(cleaned_text) or len(str(cleaned_text).strip()) < 50:
                    continue

                # Créer le document avec métadonnées
                doc = Document(
                    page_content=str(cleaned_text).strip(),
                    metadata={
                        'source': row['source'],
                        'csv_file': csv_file.name,
                        'page_index': idx,
                        'chunk_id': f"{csv_file.stem}_{idx}"
                    }
                )
                file_documents.append(doc)

            all_documents.extend(file_documents)
            total_chunks += len(file_documents)
            print(f"  ✅ {len(file_documents)} chunks extraits de {csv_file.name}")

        except Exception as e:
            print(f"  ❌ Erreur lors du traitement de {csv_file.name}: {e}")
            continue

    if not all_documents:
        print("❌ Aucun document valide trouvé dans les fichiers CSV")
        return None

    print(f"\n📊 Total: {total_chunks} chunks à indexer")

    # Créer le dossier ChromaDB s'il n'existe pas
    chroma_path = Path(chroma_dir)
    chroma_path.mkdir(exist_ok=True)

    print(f"🆕 Création d'une nouvelle base de données ChromaDB dans '{chroma_dir}'")

    # Créer une nouvelle base avec tous les documents
    print("📤 Création de la base vectorielle avec embeddings Gemini...")
    print("⏳ Cela peut prendre quelques minutes selon le nombre de documents...")

    vectorstore = Chroma.from_documents(
        documents=all_documents,
        embedding=gemini_embeddings,
        persist_directory=chroma_dir
    )

    # Persister et finaliser la base de données
    print("💾 Sauvegarde et finalisation de la base de données...")
    vectorstore.persist()

    # Créer le retriever pour les recherches
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

    print(f"\n✅ Base de données ChromaDB créée avec succès!")
    print(f"📊 Statistiques finales:")
    print(f"  • {len(csv_files)} fichiers CSV traités")
    print(f"  • {total_chunks} chunks indexés dans ChromaDB")
    print(f"  • Base de données créée dans '{chroma_dir}'")
    print(f"  • Retriever configuré pour k=5 résultats")

    return vectorstore, retriever

def test_vectorstore(retriever, test_query: str = "student performance"):
    """
    Test simple de la base vectorielle
    """
    print(f"\n🧪 Test de recherche avec la requête: '{test_query}'")

    try:
        results = retriever.get_relevant_documents(test_query)
        print(f"📋 {len(results)} résultats trouvés:")

        for i, doc in enumerate(results[:3], 1):  # Afficher seulement les 3 premiers
            print(f"\n  Résultat {i}:")
            print(f"    Source: {doc.metadata.get('source', 'N/A')}")
            print(f"    CSV: {doc.metadata.get('csv_file', 'N/A')}")
            print(f"    Contenu: {doc.page_content[:200]}...")

    except Exception as e:
        print(f"❌ Erreur lors du test: {e}")

def main():
    """
    Fonction principale
    """
    try:
        # Charger les CSV vers ChromaDB
        vectorstore, retriever = load_cleaned_csvs_to_chroma()

        if vectorstore and retriever:
            # Test optionnel
            test_vectorstore(retriever)

            # La base est maintenant prête à être utilisée
            print(f"\n🎉 Nouvelle base vectorielle ChromaDB créée!")
            print(f"💡 Vous pouvez maintenant utiliser:")
            print(f"   - 'vectorstore' pour accéder directement à la base")
            print(f"   - 'retriever' pour effectuer des recherches")
            print(f"📁 Base sauvegardée dans: ./chroma_db")

            return vectorstore, retriever
        else:
            print("❌ Échec du chargement")
            return None, None

    except Exception as e:
        print(f"❌ Erreur critique: {e}")
        return None, None

if __name__ == "__main__":
    vectorstore, retriever = main()

🔄 Initialisation du chargement des CSV vers ChromaDB...
📁 125 fichiers CSV trouvés

📄 Traitement de cleanedup_Copie de Djulovic and Li - 2013 - Towards freshman retention prediction A comparative study.csv...
  ✅ 7 chunks extraits de cleanedup_Copie de Djulovic and Li - 2013 - Towards freshman retention prediction A comparative study.csv

📄 Traitement de cleanedup_Copie de 2019 - Detecting students-at-risk in computer programming classes with learning analytics from students’ di.csv...
  ✅ 29 chunks extraits de cleanedup_Copie de 2019 - Detecting students-at-risk in computer programming classes with learning analytics from students’ di.csv

📄 Traitement de cleanedup_Copie de 2022 - On the Use of eXplainable Artificial Intelligence to Evaluate School Dropout.csv...
  ✅ 19 chunks extraits de cleanedup_Copie de 2022 - On the Use of eXplainable Artificial Intelligence to Evaluate School Dropout.csv

📄 Traitement de cleanedup_Copie de Mitra and Goldstein - 2015 - Designing early detection a

<ipython-input-13-2550768644>:114: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
<ipython-input-13-2550768644>:135: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(test_query)


📋 5 résultats trouvés:

  Résultat 1:
    Source: Rafique - 2021 - Integrating Learning Analytics and Collaborative Learning for Improving Student_s Academic Performan.pdf
    CSV: cleanedup_Rafique - 2021 - Integrating Learning Analytics and Collaborative Learning for Improving Student_s Academic Performan.csv
    Contenu: RQ2. WHICH ARE THE MOST IMPORTANT FEATURES
THAT HELP ACCURATE PREDICTION OF STUDENT’S
PERFORMANCE?
The second question determines important features to make
accurate predictions of students’ performan...

  Résultat 2:
    Source: 2018 - Exploring the High Potential Factors that Affects Students’ Academic Performance.pdf
    CSV: cleanedup_2018 - Exploring the High Potential Factors that Affects Students’ Academic Performance.csv
    Contenu: an on -line discussion forum. With the proper format data, classification and classification via clustering techniques are applied and compared. Finally, the obtained classification models are describ...

  Résultat 3:
    Sour

#Tester le RAG

In [ ]:
import os
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Optional
from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from tqdm import tqdm

def load_cleaned_csvs_to_chroma(csv_dir: str = "clean_up_test",
                               chroma_dir: str = "chroma_db",
                               batch_size: int = 50):
    """
    Charge tous les fichiers CSV nettoyés du dossier csv_dir vers ChromaDB.
    Si ChromaDB existe déjà, ajoute les nouveaux documents à l'existante.
    Sinon, crée une nouvelle base de données.

    Args:
        csv_dir: Répertoire contenant les fichiers CSV nettoyés
        chroma_dir: Répertoire de la base de données ChromaDB
        batch_size: Nombre de documents à traiter par batch
    """

    print("🔄 Initialisation du chargement des CSV vers ChromaDB...")

    # Configuration des embeddings
    gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Vérifier si le dossier CSV existe
    csv_path = Path(csv_dir)
    if not csv_path.exists():
        raise FileNotFoundError(f"Le dossier '{csv_dir}' n'existe pas.")

    # Créer le dossier ChromaDB s'il n'existe pas
    chroma_path = Path(chroma_dir)
    chroma_path.mkdir(exist_ok=True)

    # Vérifier si ChromaDB existe déjà
    existing_vectorstore = None
    existing_chunk_ids = set()

    # Vérifier la présence de fichiers ChromaDB (index, sqlite, etc.)
    chroma_files = list(chroma_path.glob("*"))
    database_exists = len(chroma_files) > 0

    if database_exists:
        try:
            print("🔍 Base ChromaDB existante détectée, chargement...")
            existing_vectorstore = Chroma(
                persist_directory=chroma_dir,
                embedding_function=gemini_embeddings
            )

            # Récupérer les chunk_ids existants pour éviter les doublons
            try:
                # Tentative de récupération des métadonnées existantes
                existing_docs = existing_vectorstore.get()
                if existing_docs and 'metadatas' in existing_docs:
                    for metadata in existing_docs['metadatas']:
                        if metadata and 'chunk_id' in metadata:
                            existing_chunk_ids.add(metadata['chunk_id'])

                print(f"📊 {len(existing_chunk_ids)} chunks existants trouvés dans la base")
            except Exception as e:
                print(f"⚠️ Impossible de récupérer les métadonnées existantes: {e}")
                print("🔄 Continuons avec une vérification basique...")

        except Exception as e:
            print(f"⚠️ Erreur lors du chargement de la base existante: {e}")
            print("🔄 Création d'une nouvelle base...")
            existing_vectorstore = None
            database_exists = False

    # Trouver tous les fichiers CSV
    csv_files = list(csv_path.glob("*.csv"))

    if not csv_files:
        print(f"❌ Aucun fichier CSV trouvé dans '{csv_dir}'")
        return existing_vectorstore.as_retriever(search_kwargs={"k": 5}) if existing_vectorstore else None

    print(f"📁 {len(csv_files)} fichiers CSV trouvés")

    # Collecter tous les nouveaux documents
    new_documents = []
    total_new_chunks = 0
    skipped_chunks = 0

    for csv_file in csv_files:
        print(f"\n📄 Traitement de {csv_file.name}...")

        try:
            # Charger le CSV
            df = pd.read_csv(csv_file)

            # Vérifier que les colonnes nécessaires existent
            if 'cleaned_page_content' not in df.columns:
                print(f"  ⚠️ Colonne 'cleaned_page_content' manquante dans {csv_file.name}")
                continue

            if 'source' not in df.columns:
                print(f"  ⚠️ Colonne 'source' manquante dans {csv_file.name}")
                continue

            # Créer les documents pour chaque ligne
            file_documents = []
            file_skipped = 0

            for idx, row in df.iterrows():
                cleaned_text = row['cleaned_page_content']

                # Ignorer les chunks vides ou très courts
                if pd.isna(cleaned_text) or len(str(cleaned_text).strip()) < 50:
                    continue

                # Générer l'ID unique pour ce chunk
                chunk_id = f"{csv_file.stem}_{idx}"

                # Vérifier si ce chunk existe déjà
                if chunk_id in existing_chunk_ids:
                    file_skipped += 1
                    continue

                # Créer le document avec métadonnées
                doc = Document(
                    page_content=str(cleaned_text).strip(),
                    metadata={
                        'source': row['source'],
                        'csv_file': csv_file.name,
                        'page_index': idx,
                        'chunk_id': chunk_id
                    }
                )
                file_documents.append(doc)

            new_documents.extend(file_documents)
            total_new_chunks += len(file_documents)
            skipped_chunks += file_skipped

            print(f"  ✅ {len(file_documents)} nouveaux chunks extraits de {csv_file.name}")
            if file_skipped > 0:
                print(f"  ⏭️ {file_skipped} chunks ignorés (déjà existants)")

        except Exception as e:
            print(f"  ❌ Erreur lors du traitement de {csv_file.name}: {e}")
            continue

    # Résumé des documents à traiter
    print(f"\n📊 Résumé:")
    print(f"  • {total_new_chunks} nouveaux chunks à ajouter")
    print(f"  • {skipped_chunks} chunks ignorés (doublons)")
    print(f"  • {len(existing_chunk_ids)} chunks déjà en base")

    # Si aucun nouveau document, retourner l'existant
    if not new_documents:
        print("ℹ️ Aucun nouveau document à ajouter")
        if existing_vectorstore:
            retriever = existing_vectorstore.as_retriever(search_kwargs={"k": 5})
            print("✅ Utilisation de la base existante")
            return existing_vectorstore, retriever
        else:
            print("❌ Aucune base existante et aucun nouveau document")
            return None, None

    # Traitement selon l'existence ou non de la base
    if database_exists and existing_vectorstore:
        print(f"➕ Ajout de {total_new_chunks} nouveaux chunks à la base existante...")
        print("⏳ Génération des embeddings et ajout à la base...")

        # Ajouter les nouveaux documents par batch
        try:
            # Ajouter tous les documents d'un coup (ChromaDB gère les batches en interne)
            existing_vectorstore.add_documents(new_documents)
            vectorstore = existing_vectorstore

        except Exception as e:
            print(f"❌ Erreur lors de l'ajout: {e}")
            print("🔄 Tentative de création d'une nouvelle base...")
            # Fallback: créer une nouvelle base avec tous les documents
            all_docs = new_documents  # On ne peut pas récupérer les anciens facilement
            vectorstore = Chroma.from_documents(
                documents=all_docs,
                embedding=gemini_embeddings,
                persist_directory=chroma_dir
            )
    else:
        print(f"🆕 Création d'une nouvelle base de données ChromaDB dans '{chroma_dir}'")
        print("📤 Création de la base vectorielle avec embeddings Gemini...")
        print("⏳ Cela peut prendre quelques minutes selon le nombre de documents...")

        vectorstore = Chroma.from_documents(
            documents=new_documents,
            embedding=gemini_embeddings,
            persist_directory=chroma_dir
        )

    # Persister et finaliser la base de données
    print("💾 Sauvegarde et finalisation de la base de données...")
    vectorstore.persist()

    # Créer le retriever pour les recherches
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

    # Statistiques finales
    total_in_db = len(existing_chunk_ids) + total_new_chunks

    print(f"\n✅ Base de données ChromaDB mise à jour avec succès!")
    print(f"📊 Statistiques finales:")
    print(f"  • {len(csv_files)} fichiers CSV traités")
    print(f"  • {total_new_chunks} nouveaux chunks ajoutés")
    print(f"  • {total_in_db} chunks totaux dans la base")
    print(f"  • Base de données dans '{chroma_dir}'")
    print(f"  • Retriever configuré pour k=5 résultats")

    return vectorstore, retriever

def get_database_stats(chroma_dir: str = "chroma_db") -> dict:
    """
    Récupère les statistiques de la base ChromaDB existante
    """
    try:
        gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        vectorstore = Chroma(
            persist_directory=chroma_dir,
            embedding_function=gemini_embeddings
        )

        # Récupérer les informations de la base
        docs_info = vectorstore.get()

        stats = {
            'total_documents': len(docs_info['ids']) if docs_info['ids'] else 0,
            'csv_files': set(),
            'sources': set()
        }

        if docs_info.get('metadatas'):
            for metadata in docs_info['metadatas']:
                if metadata:
                    if 'csv_file' in metadata:
                        stats['csv_files'].add(metadata['csv_file'])
                    if 'source' in metadata:
                        stats['sources'].add(metadata['source'])

        stats['unique_csv_files'] = len(stats['csv_files'])
        stats['unique_sources'] = len(stats['sources'])

        return stats

    except Exception as e:
        return {'error': str(e)}

def test_vectorstore(retriever, test_query: str = "student performance"):
    """
    Test simple de la base vectorielle
    """
    print(f"\n🧪 Test de recherche avec la requête: '{test_query}'")

    try:
        results = retriever.get_relevant_documents(test_query)
        print(f"📋 {len(results)} résultats trouvés:")

        for i, doc in enumerate(results[:3], 1):  # Afficher seulement les 3 premiers
            print(f"\n  Résultat {i}:")
            print(f"    Source: {doc.metadata.get('source', 'N/A')}")
            print(f"    CSV: {doc.metadata.get('csv_file', 'N/A')}")
            print(f"    Chunk ID: {doc.metadata.get('chunk_id', 'N/A')}")
            print(f"    Contenu: {doc.page_content[:200]}...")

    except Exception as e:
        print(f"❌ Erreur lors du test: {e}")

def main():
    """
    Fonction principale
    """
    try:
        # Afficher les stats de la base existante si elle existe
        chroma_dir = "chroma_db"
        if Path(chroma_dir).exists() and list(Path(chroma_dir).glob("*")):
            print("📊 Statistiques de la base existante:")
            stats = get_database_stats(chroma_dir)
            if 'error' not in stats:
                print(f"  • {stats['total_documents']} documents en base")
                print(f"  • {stats['unique_csv_files']} fichiers CSV uniques")
                print(f"  • {stats['unique_sources']} sources uniques")
            else:
                print(f"  ⚠️ Erreur lors de la lecture des stats: {stats['error']}")

        # Charger les CSV vers ChromaDB (ajout incrémental)
        vectorstore, retriever = load_cleaned_csvs_to_chroma()

        if vectorstore and retriever:
            # Test optionnel
            test_vectorstore(retriever)

            # La base est maintenant prête à être utilisée
            print(f"\n🎉 Base vectorielle ChromaDB mise à jour!")
            print(f"💡 Vous pouvez maintenant utiliser:")
            print(f"   - 'vectorstore' pour accéder directement à la base")
            print(f"   - 'retriever' pour effectuer des recherches")
            print(f"📁 Base sauvegardée dans: ./chroma_db")

            return vectorstore, retriever
        else:
            print("❌ Échec du chargement")
            return None, None

    except Exception as e:
        print(f"❌ Erreur critique: {e}")
        return None, None

if __name__ == "__main__":
    vectorstore, retriever = main()

📊 Statistiques de la base existante:
  • 92 documents en base
  • 7 fichiers CSV uniques
  • 7 sources uniques
🔄 Initialisation du chargement des CSV vers ChromaDB...
🔍 Base ChromaDB existante détectée, chargement...
📊 92 chunks existants trouvés dans la base
📁 19 fichiers CSV trouvés

📄 Traitement de cleanedup_Copie de Djulovic and Li - 2013 - Towards freshman retention prediction A comparative study.csv...
  ✅ 0 nouveaux chunks extraits de cleanedup_Copie de Djulovic and Li - 2013 - Towards freshman retention prediction A comparative study.csv
  ⏭️ 7 chunks ignorés (déjà existants)

📄 Traitement de cleanedup_Copie de 2019 - Detecting students-at-risk in computer programming classes with learning analytics from students’ di.csv...
  ✅ 0 nouveaux chunks extraits de cleanedup_Copie de 2019 - Detecting students-at-risk in computer programming classes with learning analytics from students’ di.csv
  ⏭️ 29 chunks ignorés (déjà existants)

📄 Traitement de cleanedup_Copie de 2022 - On the Use

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, top_p =1, max_retries=3)

In [ ]:
#This question is outside the scope of the provided documents.


LLM_CONTEXT = """
Prompt:

You are a research question-answering assistant with access to a curated set of academic papers on student performance.

You must strictly follow these instructions:

Answer only based on the provided context. Never speculate or rely on external knowledge.

If the answer is not found in the context, respond exactly:

The answer is not available in the provided documents.

If the question is off-topic (not about student performance), respond exactly:

Your question is so fucking stupid, you piece of shit. I'm here only to answer questions about student performances.

Keep your answer concise, using a maximum of five sentences.

Always cite your source(s) clearly using the document filename or title.

Format your answer in Markdown.

Respond in the same language as the question.

Example

Question:
Quels sont les facteurs les plus corrélés à la réussite des étudiants selon les études disponibles ?

Answer:
Plusieurs études indiquent que l’assiduité, le niveau socio-économique et la motivation intrinsèque sont des facteurs fortement corrélés à la réussite des étudiants. Par exemple, le document student_success_analysis.pdf met en avant l’impact positif d’une forte participation en classe.
Source : student_success_analysis.pdf

Input:

Question: {question}
Context: {context}
Answer:
"""
llm_prompt = PromptTemplate.from_template(LLM_CONTEXT)

print(llm_prompt)


input_variables=['context', 'question'] input_types={} partial_variables={} template="\nPrompt:\n\nYou are a research question-answering assistant with access to a curated set of academic papers on student performance.\n\nYou must strictly follow these instructions:\n\nAnswer only based on the provided context. Never speculate or rely on external knowledge.\n\nIf the answer is not found in the context, respond exactly:\n\nThe answer is not available in the provided documents.\n\nIf the question is off-topic (not about student performance), respond exactly:\n\nYour question is so fucking stupid, you piece of shit. I'm here only to answer questions about student performances.\n\nKeep your answer concise, using a maximum of five sentences.\n\nAlways cite your source(s) clearly using the document filename or title.\n\nFormat your answer in Markdown.\n\nRespond in the same language as the question.\n\nExample\n\nQuestion:\nQuels sont les facteurs les plus corrélés à la réussite des étudiant

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content = ""),
    AIMessage(content = ""),
]
response = client.invoke(messages)
print(reponse.content)


AttributeError: 'Client' object has no attribute 'invoke'

In [ ]:
def format_docs_with_sources(docs):
    """Format les documents avec leurs sources pour le contexte RAG"""
    formatted_docs = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'N/A')
        # Formatage avec numéro de document et source
        formatted_doc = f"Document {i} (Source: {source}):\n{doc.page_content}"
        formatted_docs.append(formatted_doc)

    return "\n\n" + "="*80 + "\n\n".join(formatted_docs)

def get_sources_used(prompt_question, retriever, top_k=3):
    """Récupère et affiche les sources utilisées pour une question"""
    results = retriever.get_relevant_documents(prompt_question)
    sources_used = []

    print(f"\nSources utilisées pour répondre à la question (Top {top_k}):")
    print("-" * 60)

    for i, doc in enumerate(results[:top_k], 1):
        source = doc.metadata.get('source', 'N/A')
        sources_used.append(source)
        print(f"{i}. {source}")
    return sources_used

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs_with_sources, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
#prompt_question = "As a teacher, how can I use learning analytics to collect data for tracking, monitoring, and enhancing students’ performance?"
'''
prompt_question = "tu t'appelles comment gros?"

print(rag_chain.invoke(prompt_question))
sources = get_sources_used(prompt_question, retriever)
'''
t = True
i = 0
while t:
  i +=1
  prompt_question = str(input("pose ta question: "))
  reponse = rag_chain.invoke(prompt_question)
  if "fucking" not in str(reponse):
    print(reponse)
    sources = get_sources_used(prompt_question, retriever)
    t = False
  else:
    if i < 5:
      print("try again: out of context question / rententez: question hors sujet")
    elif i < 10:
      print("the subject is student's performance. Please, be serious")
    else:
      print(reponse)

KeyboardInterrupt: Interrupted by user

In [ ]:
results = retriever.get_relevant_documents(prompt_question)

for i, doc in enumerate(results[:3], 1):  # Afficher seulement les 3 premiers
            source = doc.metadata.get('source', 'N/A')

2017 - Using Learning Analytics to Predict Students’ Performance in Moodle Learning Management System A Ca.pdf
cleanedup_2017 - Using Learning Analytics to Predict Students’ Performance in Moodle Learning Management System A Ca.csv
2013 - Correlation between Course Tracking Variables and Academic Performance in Blended Online Courses.pdf
cleanedup_2013 - Correlation between Course Tracking Variables and Academic Performance in Blended Online Courses.csv
2017 - Using Learning Analytics to Predict Students’ Performance in Moodle Learning Management System A Ca.pdf
cleanedup_2017 - Using Learning Analytics to Predict Students’ Performance in Moodle Learning Management System A Ca.csv


As a teacher, you can use learning analytics to collect data for tracking, monitoring, and enhancing students’ performance by:\n\n*   **Using diverse data sources**: Combine data from various sources such as student characteristics, prior academic history, programming laboratory work, and logged interactions with online and offline resources ( "In this work, we propose to combine general data sources at a Higher Education Institution and to build predictive models that are able to identify students in need of assistance.").\n*   **Analyzing student digital footprints**: Leverage data from platforms managing student registration, custom learning platforms for programming submissions, and Learning Management Systems to gather clickstream data ("We identiﬁed three data sources that researchers and data scientists are often able to leverage to model student interaction, engagement and effort in computer program-ming or laboratory-intensive courses in order to build models which achieve good predictive performance and these are straightforward to store and leverage.").\n*   **Implementing predictive models**: Build predictive models using student characteristics, academic history, programming lab work, and interactions with online resources to identify students who need assistance ("We use student characteristics, prior academic history, students’ programming laboratory work, and all logged interactions between students’ ofﬂine and online resources.").\n*   **Providing adaptive feedback**: Generate personalized feedback based on each student’s progression and provide guidance when needed ("allow us to generate adaptive feedback to each student according to each student’s progression and provide guidance when in need.").\n*   **Monitoring student activities**: Track student activities through various methods such as Group Scribbles, MTClassroom, and the Formative Assessment with Computational Technology (FACT) system ("Examples include the collaborative conﬂuence working space or streamlining digitalizing physical artifacts or mobile devices.").

## Application streamlit

#### Paramètre à renseigner pour lancer l'app streamit (prochaine cellule)

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.125.151.28


#### App Streamlit (Voir fichier app.py pour modifs)

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.151.28:8501

Need to install the following packages:
  localtunnel
Ok to proceed? (y) y
your url is: https://honest-mice-remain.loca.lt


#TODO LIST

##Done
- faire truc Francais/Anglais DONE je pense
- sources (s'assurer que c'est okey) DONE
- clean up tous les pdfs et finaliser la ChromaDB DONE

##reste à faire
- few shot prompting "donner des exemples"
- structurer la réponse
- streamlit
- évaluer la performance du rag / tester la réponse du rag aux questions posées
- continuer rapport
- préparer la "soutenance"